Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

##  Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score 
from scipy.stats import multivariate_normal
from scipy.spatial import distance
from scipy import stats

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Изучим данные на пропуски

In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Проверяем дубликаты

In [6]:
df.duplicated().sum()

153

Удалим дубликаты

In [7]:
df.drop_duplicates(inplace = True)

Для выбора модели обучения оценим нащу целевую переменную. 

In [8]:
df['Страховые выплаты'].value_counts()

0    4284
1     423
2     114
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

Целевая переменная дискретная количественная, для ее предсказания решается задача регресии. 

## Умножение матриц

### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Нет, не изменится



**Обоснование:** ...Пусть у нас есть матрица $X$ размером $mxn$, где $n$ - число признаков , а  $m$ - число строк. Предположим, что существует  обратимая матрица $P$, размером $nxn$. Тогда мы можем изменить наш датафрейм (ту самую матрицу $X$) умножив на $P$ , а затем вернуть первоначальные данные к исходному виду, умножив результат на обратную матрицу $P^{-1}$

А теперь необходимо проверить будет ли модель регресии предсказывать результат такой же , как если бы не была осуществлена трансформация данных. 

Подставим в формулу $a = Xw$,  значения новую матрицу $XP$, тогда получим выражение:

$$
a' = X'w' = XP((XP^T)XP))^{-1}(XP)^Ty = XP(P^TX^TXP)^{-1}P^TX^Ty = XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty = 
 XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = XE(X^TX)^{-1}EX^Ty = X(X^TX)^{-1}X^Ty =a
$$

Таким образом получается , что вектор предсказаний трансорфмированной  и первичной матриц один и тот же. 

## Алгоритм преобразования

**Алгоритм**

Разобьем датафрейм на обучающий набор данных и целевой признак. 

Подберем кодирующую квадратную  матрицу размером равным количеству  признаков в обучающем наборе. Убедимся в ее обратимости.

Трансформируем обучающий набор умножив его на  кодирующую матрицу. 

Сформируем предсказания по трансформированным данным. 

Сформируем предсказания по оригинальным данным.

Проверим расстояние между векторами предсказаний трансформированной матрицы и оригинальной 

Сверим качество предсказания модели в обоих случаях.

Востановим данные умножив трансформированную матрицу на обратную кодирующей. 

Сверим первоначальные данные с востановленными. 



**Обоснование**

Как мы показали в п.2 матрицу можно трансфоримировать из состояния одного состояния в другое, а за тем вернуть обратно. При этом мы можем создать  бесконечное число кодирующих матриц. Что означает что определить первоночальную матрицу по  трансформированной практически невозможно. Данные будут надежно защищены. 

А теперь проверим алгоритм на практике

## Проверка алгоритма

Разбиваем признаки на целевой и обучающие

In [17]:
features =  df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

Посчитаем точность качество модели на оригинальной матрице

In [37]:
model_before = LinearRegression()
model_before.fit(features, target)
predicted_before = model_before.predict(features)
print("R2 =", r2_score(target,predicted_before))

R2 = 0.4302010044852067


Сгенерим кодирующую матрицу

In [27]:
P = np.random.normal(size=(4, 4))

In [28]:
P

array([[-0.08256758,  0.45354482,  0.8623544 , -0.53372257],
       [-0.80969963, -0.26129152,  0.07232908,  0.52231616],
       [-0.91653753,  1.05490423, -2.66304238,  0.50930625],
       [-1.68635864, -0.91010943, -0.20719933,  0.61311329]])

Прокрутив последнии две строчки кода несколько раз , убеждаемся ,  что кодирующая матрица всегда разная. 

Убедимся в обратимости кодирующей матрицы

In [29]:
P @ np.linalg.inv(P)

array([[ 1.00000000e+00,  2.32879733e-16, -4.48077810e-17,
        -4.95494650e-17],
       [ 5.40452885e-17,  1.00000000e+00, -1.00951072e-17,
        -1.58897256e-17],
       [ 1.58794630e-16,  2.97907661e-16,  1.00000000e+00,
        -7.80472293e-17],
       [ 6.75653364e-17,  9.22149617e-17, -1.87233670e-17,
         1.00000000e+00]])

Главная диагональ матрицы это единицы , остальные данные очень близки к нулю. Матрица обратима

Трансформируем оригинальную матрицу

In [35]:
trans_features = features @ P

In [32]:
trans_features.head()

,0,1,2,3
0,-45495.228269,52312.080218,-132083.281466,25283.084503
1,-34867.358812,40073.431165,-101192.490552,19378.277271
2,-19270.769491,22145.411345,-55921.792465,10710.578482
3,-38239.991554,43982.198989,-111047.762789,21250.265616
4,-23944.383780,27526.137747,-69502.518584,13306.984333


Матрица выглядит как абсолютно случайный набор данных.  

Сгенерим вектор предсказаний  и померим качество предсказаний

In [36]:
model_after = LinearRegression()
model_after.fit(trans_features, target)
predicted_after = model_after.predict(trans_features)
print("R2 =", r2_score(target,predicted_after))

R2 = 0.43020100448522025


Проверим расстояние между  векторами предсказаний 

In [38]:
d = distance.euclidean(predicted_after, predicted_before)

In [39]:
print(d)

8.762330975230632e-11


Модели предсказывают одинаково, расстояние между векторами предсказаний $0.8^{-10}$ (практически ноль)

Сможем ли вернуть данные в первоначальный вид?

In [41]:

rebuilt = trans_features @ np.linalg.inv(P); rebuilt.head()



,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-3.227300e-12,46.0,38000.0,1.000000e+00
2,3.005860e-12,29.0,21000.0,-1.212063e-12
3,9.820252e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-2.491688e-12


In [42]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Данные востановлены. 

## Вывод

Проведена процедура кодирования данных. Мы убедились в надежности кодирования, данные выглядят как случайные набор данных. 
Трансформированные и оригинальные данные дают идентичные предсказания. Качество предсказания в обоих случаях одинаково. 
Из трансформированных данных востановленные оригинальные с минимальной точностью $1^{-12}$

Алгоритм работает!